In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
cptac warning: Your version of cptac (0.8.0) is out-of-date. Latest is 0.8.1. Please run 'pip install --upgrade cptac' to update it. (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 10)


In [2]:
en = cptac.Endometrial()
br = cptac.Brca()
cl = cptac.Colon()

In [8]:
#lets check how many missense mutations are in proteomics. Check it against out function as well
mut_type = cl.get_genotype_all_vars("PIK3CA")
mut = mut_type.loc[mut_type["Mutation"]!="Wildtype_Tumor"]
len(mut)
len(mut.loc[mut["Mutation"] == "nonsynonymous SNV"])


proteomics = cl.get_proteomics(tissue_type="tumor")
proteomics

mutat = cl.get_somatic_mutation()
mut = mutat.loc[mutat["Gene"] == "PIK3CA"]
len(mut.loc[mut["Mutation"]== "nonsynonymous SNV"])
mut.loc[mut["Mutation"]== "nonsynonymous SNV"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Gene,Mutation,Location
Patient_ID,,,
01CO022,PIK3CA,nonsynonymous SNV,Q981R
05CO003,PIK3CA,nonsynonymous SNV,C420R
05CO003,PIK3CA,nonsynonymous SNV,H1047Y
05CO006,PIK3CA,nonsynonymous SNV,E103G
05CO028,PIK3CA,nonsynonymous SNV,E545K
05CO032,PIK3CA,nonsynonymous SNV,H1047R
05CO050,PIK3CA,nonsynonymous SNV,E85G
06CO002,PIK3CA,nonsynonymous SNV,M1043I
09CO005,PIK3CA,nonsynonymous SNV,H1047R


In [9]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = cl.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = cl.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)

joined
# #join wildtype proteomics to missense mutation proteomics 
# wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
# missense = joined.loc[joined.Mutation == 'nonsynonymous SNV']
# colon_both = missense.append(wildtype)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Mutation,Location,Mutation_Status,A1BG,A1CF,A2M,AAAS,AACS,AAGAB,AAK1,...,ZNHIT6,ZNRD1,ZNRF2,ZPR1,ZRANB2,ZW10,ZWILCH,ZWINT,ZYX,ZZEF1
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01CO001,Wildtype_Tumor,No_mutation,Wildtype_Tumor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01CO005,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.100,0.3180,-0.4870,0.0995,0.1550,0.1690,0.0653,...,NaN,NaN,0.0384,0.0221,0.2500,0.0869,0.0331,NaN,-0.7330,-0.2650
01CO006,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.120,-0.4410,-0.3470,-0.0029,0.0957,0.3960,-0.0363,...,NaN,0.324,NaN,-0.3940,0.0846,0.0010,-0.3450,NaN,-0.6580,0.0052
01CO008,Amplification,Amplification,Single_mutation,-1.200,0.1600,-1.8500,0.1190,-0.0924,0.0187,-0.2140,...,NaN,NaN,-0.3840,-0.1680,0.3570,-0.3250,0.3490,NaN,-0.8210,0.2000
01CO013,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.890,0.1120,-0.3290,0.6700,0.1160,0.3130,-0.2380,...,NaN,NaN,NaN,-0.0656,-0.2950,0.4630,-0.4480,0.0493,-0.9040,-0.1580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21CO007,Amplification,Amplification,Single_mutation,-0.975,0.3980,-1.1500,0.5730,-0.1280,0.0600,-0.0559,...,NaN,NaN,0.2460,0.0047,0.1650,0.1030,NaN,NaN,-0.7520,-0.2410
22CO004,Wildtype_Tumor,No_mutation,Wildtype_Tumor,-1.280,0.5180,-0.8720,0.2580,0.4240,0.2980,-0.1500,...,NaN,NaN,0.4490,-0.0616,0.3850,0.1160,-0.0437,NaN,-0.8840,0.0088
22CO006,Wildtype_Tumor,No_mutation,Wildtype_Tumor,0.515,-1.2100,-0.0283,0.1780,-0.3030,0.1500,0.3990,...,NaN,NaN,-0.5610,0.4740,0.0836,-0.2510,0.1760,NaN,0.0584,-0.3010


In [12]:
joined["Mutation"].unique()

array(['Wildtype_Tumor', 'Amplification', 'nonframeshift substitution',
       'nonsynonymous SNV', 'Deletion', 'frameshift insertion'],
      dtype=object)

In [20]:
#ENDO
mut_type = en.get_genotype_all_vars("PIK3CA")
mut = mut_type.loc[mut_type["Mutation"]!="Wildtype_Tumor"]
len(mut)
len(mut.loc[mut["Mutation"] == "Missense_Mutation"])


proteomics = en.get_proteomics(tissue_type="tumor")
proteomics

mutat = en.get_somatic_mutation()
mut = mutat.loc[mutat["Gene"] == "PIK3CA"]
len(mut.loc[mut["Mutation"]== "Missense_Mutation"])
mut.loc[mut["Mutation"]== "Missense_Mutation"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Gene,Mutation,Location
Patient_ID,,,
C3L-00006,PIK3CA,Missense_Mutation,p.E545K
C3L-00032,PIK3CA,Missense_Mutation,p.E545K
C3L-00139,PIK3CA,Missense_Mutation,p.Q546P
C3L-00143,PIK3CA,Missense_Mutation,p.R115L
C3L-00156,PIK3CA,Missense_Mutation,p.L628R
C3L-00156,PIK3CA,Missense_Mutation,p.T1025A
C3L-00161,PIK3CA,Missense_Mutation,p.R38C
C3L-00161,PIK3CA,Missense_Mutation,p.V344M
C3L-00362,PIK3CA,Missense_Mutation,p.H1047R


In [29]:
#  Step 1 - Create dataframe in order to do comparisons with wrap_ttest
proteomics = en.get_proteomics(tissue_type="tumor")

prot_list = list(proteomics.columns)

mut_type = en.get_genotype_all_vars("PIK3CA")


joined = mut_type.join(proteomics)

joined
# #join wildtype proteomics to missense mutation proteomics 
# wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]
missense = joined.loc[joined.Mutation == 'Missense_Mutation']
# colon_both = missense.append(wildtype)
len(missense)
# len(joined)
missense[["Mutation", "PIK3CA"]]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


Name,Mutation,PIK3CA
Patient_ID,,
C3L-00006,Missense_Mutation,-0.450000
C3L-00032,Missense_Mutation,-0.178000
C3L-00139,Missense_Mutation,0.282000
C3L-00143,Missense_Mutation,-0.071400
C3L-00156,Missense_Mutation,-0.743000
C3L-00161,Missense_Mutation,-0.274000
C3L-00362,Missense_Mutation,-0.469000
C3L-00563,Missense_Mutation,-0.472000
C3L-00586,Missense_Mutation,-0.535000


In [ ]:
Frame_Shift_Del', 'Frame_Shift_Ins', 'Nonsense_Mutation', 'Nonstop_Mutation'

In [ ]:
#Maybe we should look at a particular gene. 
